In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import regularizers
print(os.listdir("/Users/fadhlihajar/Desktop/ArSLDatabase"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['train.pdf', '.DS_Store', 'test', 'train2', 'train3', 'train']


In [75]:
print(train_dir)

/Users/fadhlihajar/Desktop/ArSLDatabase/train2


In [2]:
train_dir = '/Users/fadhlihajar/Desktop/ArSLDatabase/train3'
test_dir = '/Users/fadhlihajar/Desktop/ArSLDatabase/test'

In [5]:
def load_unique():
    size_img = 64,64
    images_for_plot = []
    labels_for_plot = []
    for folder in os.listdir(train_dir):
        for file in os.listdir(train_dir + '/' + folder):
            filepath = train_dir + '/' + folder + '/' + file
            image = cv2.imread(filepath)
            final_img = cv2.resize(image, size_img)
            final_img = cv2.cvtColor(final_img, cv2.COLOR_BGR2RGB)
            images_for_plot.append(final_img)
            labels_for_plot.append(folder)
            break
    return images_for_plot, labels_for_plot

images_for_plot, labels_for_plot = load_unique()
print("unique_labels = ", labels_for_plot)

unique_labels =  ['bb', 'waw', 'al', 'nun', 'aleff']


In [6]:
labels_dict = {'bb':0,'waw':1,'al':2,'nun':3,'aleff':4}

def load_data():
    images = []
    labels = []
    size = 64,64
    print("LOADING DATA FROM : ",end = "")
    for folder in os.listdir(train_dir):
        print(folder, end = ' | ')
        for image in os.listdir(train_dir + "/" + folder):
            temp_img = cv2.imread(train_dir + '/' + folder + '/' + image)
            temp_img = cv2.resize(temp_img, size)
            images.append(temp_img)
            if folder == 'bb':
                labels.append(labels_dict['bb'])
            elif folder == 'waw':
                labels.append(labels_dict['waw'])
            elif folder == 'al':
                labels.append(labels_dict['al'])
            elif folder == 'nun':
                labels.append(labels_dict['nun'])
            elif folder == 'aleff':
                labels.append(labels_dict['aleff'])
    
    images = np.array(images)
    images = images.astype('float32')/255.0
    
    labels = keras.utils.to_categorical(labels)   #one-hot encoding
    
    X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.1)
    
    print()
    print('Loaded', len(X_train),'images for training,','Train data shape =',X_train.shape)
    print('Loaded', len(X_test),'images for testing','Test data shape =',X_test.shape)
    
    return X_train, X_test, Y_train, Y_test

In [7]:
X_train, X_test, Y_train, Y_test = load_data()

LOADING DATA FROM : bb | waw | al | nun | aleff | 
Loaded 6043 images for training, Train data shape = (6043, 64, 64, 3)
Loaded 672 images for testing Test data shape = (672, 64, 64, 3)


In [8]:
def build_model():
    
    model = Sequential()
    
    model.add(Conv2D(64, kernel_size = 3, padding = 'same', activation = 'relu', input_shape = (64,64,3)))
    model.add(Conv2D(32, kernel_size = 3, padding = 'same', strides = 2, activation = 'relu'))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(32, kernel_size = 3, padding = 'same', activation = 'relu'))
    model.add(Conv2D(64, kernel_size = 3, padding = 'same', strides = 2, activation = 'relu'))
    model.add(Dropout(0.5))
    
    model.add(Conv2D(128, kernel_size = 3, padding = 'same', activation = 'relu'))
    model.add(Conv2D(256, kernel_size = 3, padding = 'same', strides = 2 , activation = 'relu'))
    model.add(MaxPool2D(3))
    
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dense(5, activation = 'softmax'))
    
    model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ["accuracy"])
    
    print("MODEL CREATED")
    model.summary()
    
    return model

In [9]:
def fit_model():
    history = model.fit(X_train, Y_train, batch_size = 64, epochs = 5, validation_split = 0.1)
    return history

In [11]:
model = build_model()

MODEL CREATED
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 64, 64, 64)        1792      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 32)        18464     
_________________________________________________________________
dropout_4 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_5 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 16, 

In [12]:
model_history = fit_model()

Train on 5438 samples, validate on 605 samples
Epoch 1/5
5438/5438 [==============================] - 39s 7ms/step - loss: 1.1595 - accuracy: 0.5143 - val_loss: 1.4696 - val_accuracy: 0.5884
Epoch 2/5
5438/5438 [==============================] - 35s 7ms/step - loss: 0.5909 - accuracy: 0.7707 - val_loss: 1.3134 - val_accuracy: 0.7719
Epoch 3/5
5438/5438 [==============================] - 36s 7ms/step - loss: 0.3918 - accuracy: 0.8580 - val_loss: 0.7953 - val_accuracy: 0.8793
Epoch 4/5
5438/5438 [==============================] - 36s 7ms/step - loss: 0.3281 - accuracy: 0.8841 - val_loss: 0.4214 - val_accuracy: 0.9190
Epoch 5/5
5438/5438 [==============================] - 36s 7ms/step - loss: 0.2281 - accuracy: 0.9152 - val_loss: 0.2818 - val_accuracy: 0.9174


In [15]:
if model_history:
    print('Final Accuracy: {:.2f}%'.format(model_history.history['accuracy'][4] * 100))
    print('Validation Set Accuracy: {:.2f}%'.format(model_history.history['accuracy'][4] * 100))

Final Accuracy: 91.52%
Validation Set Accuracy: 91.52%
